In [9]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import sys
sys.path.append('../..')

In [11]:
import asyncio
import logging

from app.redis_transcribe.connection import get_redis_client
from app.settings import settings
from app.services.transcription.processor import Processor




redis_client = await get_redis_client(settings.redis_host, settings.redis_port,settings.redis_password)

processor = Processor(redis_client,max_length=30)



2025-02-18 14:57:46,303 - INFO - Attempting Redis connection to redis:6379
2025-02-18 14:57:46,304 - INFO - Redis password is configured
2025-02-18 14:57:46,306 - INFO - Successfully connected to Redis server


In [12]:
from app.services.audio.redis_models import Transcript

In [13]:
import pandas as pd
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000
pd.options.display.max_colwidth = 1000

In [14]:
from app.services.audio.redis_models import Connection, Meeting, Transcriber

In [29]:
# Add debug logging before getting transcripts
logging.basicConfig(level=logging.DEBUG)
logger = logging.getLogger(__name__)

# Get and inspect raw data first
raw_transcripts = await redis_client.keys('transcript:*')
logger.debug(f"Found transcript keys: {raw_transcripts}")

df = await Transcript.get_all_transcripts_df(redis_client)
logger.debug(f"DataFrame columns: {df.columns}")
logger.debug(f"DataFrame shape: {df.shape}")
df.tail(10)

50


,content,start_timestamp,end_timestamp,speaker,confidence,segment_id,meeting_id,server_timestamp,transcription_timestamp,present_user_ids,partially_present_user_ids
32,А там нет.,2025-02-13 15:35:09.478000+00:00,2025-02-13 15:35:10.738000+00:00,Sergey Ryabenko,0.414286,49,the-zdjv-byg,2025-02-13 15:32:16+00:00,2025-02-18 14:57:49.949662+00:00,[00000000-0000-4000-a000-000000000001],[]
33,Данные,2025-02-13 15:35:11.378000+00:00,2025-02-13 15:35:11.998000+00:00,None,0.000000,50,the-zdjv-byg,2025-02-13 15:32:16+00:00,2025-02-18 14:57:49.949662+00:00,[00000000-0000-4000-a000-000000000001],[]
34,в одном месте все хранятся.,2025-02-13 15:35:11.998000+00:00,2025-02-13 15:35:13.398000+00:00,Dmitriy Grankin,0.284286,51,the-zdjv-byg,2025-02-13 15:32:16+00:00,2025-02-18 14:57:49.949662+00:00,[00000000-0000-4000-a000-000000000001],[]
35,Это,2025-02-13 15:35:13.498000+00:00,2025-02-13 15:35:13.678000+00:00,Dmitriy Grankin,1.000000,52,the-zdjv-byg,2025-02-13 15:32:16+00:00,2025-02-18 14:57:49.949662+00:00,[00000000-0000-4000-a000-000000000001],[]
0,месте все хранятся там только моделька она только она принимает звук выдает транскрипты,2025-02-13 15:35:13.672000+00:00,2025-02-13 15:35:19.952000+00:00,Dmitriy Grankin,1.000000,53,the-zdjv-byg,2025-02-13 15:32:16+00:00,2025-02-18 14:57:53.569062+00:00,[00000000-0000-4000-a000-000000000001],[]
1,больше ничего не делаю она она данные вообще не хранит окей тогда подожди получается например,2025-02-13 15:35:19.952000+00:00,2025-02-13 15:35:31.812000+00:00,Dmitriy Grankin,0.341315,54,the-zdjv-byg,2025-02-13 15:32:16+00:00,2025-02-18 14:57:53.569062+00:00,[00000000-0000-4000-a000-000000000001],[]
2,я запускаю запись стучится наш экстенджи на endpoint для отправки аудио был один таки остался,2025-02-13 15:35:31.812000+00:00,2025-02-13 15:35:46.272000+00:00,Sergey Ryabenko,0.773721,55,the-zdjv-byg,2025-02-13 15:32:16+00:00,2025-02-18 14:57:53.569062+00:00,[00000000-0000-4000-a000-000000000001],[]
3,один просто он в облака переехал и общается с микросервис ну и получается что когда я отправляю,2025-02-13 15:35:46.272000+00:00,2025-02-13 15:35:57.172000+00:00,Dmitriy Grankin,0.642202,56,the-zdjv-byg,2025-02-13 15:32:16+00:00,2025-02-18 14:57:53.569062+00:00,[00000000-0000-4000-a000-000000000001],[]
4,аудио то этот контроллер находит ну как бы на,2025-02-13 15:35:57.172000+00:00,2025-02-13 15:36:03.172000+00:00,Sergey Ryabenko,1.000000,57,the-zdjv-byg,2025-02-13 15:32:16+00:00,2025-02-18 14:57:53.569062+00:00,[00000000-0000-4000-a000-000000000001],[]
5,создает сессию и начинает туда отправлять все аудио в этой в этом звонке,2025-02-13 15:36:03.572000+00:00,2025-02-13 15:36:12.572000+00:00,Sergey Ryabenko,1.000000,58,the-zdjv-byg,2025-02-13 15:32:16+00:00,2025-02-18 14:57:53.569062+00:00,[00000000-0000-4000-a000-000000000001],[]
